<a href="https://colab.research.google.com/github/mvgarcia/Reto-WJ/blob/main/modelo_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Preparación de los datos

Importamos drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos las librerías necesarias

In [ ]:
from glob import glob
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch.autograd import Variable

Cargamos los datos de entrenamiento y de testeo

In [ ]:
train_files = glob('/content/drive/MyDrive/Reto/data/train/*/*.png')
test_files = glob('/content/drive/MyDrive/Reto/data/test/*/*.png')

train_files[0]

'/content/drive/MyDrive/Reto/data/train/0/43.png'

Ordenamos ahora, aunque los datos no estén necesariamente organizados, las listas de forma aleatoria para evitar sesgos

In [ ]:
np.random.shuffle(train_files)
np.random.shuffle(test_files)

len(train_files), len(test_files)

(11340, 1256)

Ya que vamos a utilizar ResNet, al momento de cargar los datos en el modelo debemos primero reescalarlos a la entrada de la CNN (244,244,3). Ya que todas las imágenes ya son RGB, unicamente debemos reescalar el tamaño. También normalizamos el dataset utilizando la media y la mediana, algo hecho para ayudar al modelo a converger más rápidamente.

In [ ]:
data_transform = transforms.Compose([
                 transforms.Resize((224, 224)),
                 ])

data_transform_grayscale = transforms.Compose([
                 transforms.Grayscale(3),
                 transforms.Resize((224, 224)),
                 ])

Cargamos ahora los datos de entrenamiento

In [ ]:
N_train = len(train_files)
X_train = []
Y_train = []


for i, train_file in tqdm(enumerate(train_files)):
  Y_train.append(int(train_file.split('/')[7]))
  try:
    X_train.append(np.array(data_transform(Image.open(train_file))).reshape(224,224,3))
  except:
    X_train.append(np.array(data_transform_grayscale(Image.open(train_file))).reshape(224,224,3))

0it [00:00, ?it/s]

In [ ]:
N_test = len(test_files)
X_test = []
Y_test = []


for i, test_file in tqdm(enumerate(test_files)):
  Y_test.append(int(test_file.split('/')[7]))
  try:
    X_test.append(np.array(data_transform(Image.open(test_file))).reshape(224,224,3))
  except:
    X_test.append(np.array(data_transform_grayscale(Image.open(test_file))).reshape(224,224,3))

0it [00:00, ?it/s]

Convertimos estas listas con los datos a tensores de torch

In [ ]:
X_train = Variable(torch.from_numpy(np.array(X_train))).float()
Y_train = Variable(torch.from_numpy(np.array(Y_train))).long()

X_test = Variable(torch.from_numpy(np.array(X_test))).float()
Y_test = Variable(torch.from_numpy(np.array(Y_test))).long()

X_train.data.size()

torch.Size([11340, 224, 224, 3])

Y finalmente creamos el dataloader

In [ ]:
batch_size = 32

train_ds = torch.utils.data.TensorDataset(X_train, Y_train)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)

##Entrenamiento del modelo

Cargamos ResNet de la libreria de Torch

In [ ]:
import torch

model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model

Congelamos los pesos para que no se actualicen

In [ ]:
for p in model.parameters():
    p.requires_grad = False

#--- Definimos el número de clases
out_dim = 6

#--- Reescribimos la nueva capa de salida con el nuevo dataset
model.fc = torch.nn.Sequential(
  torch.nn.Linear(model.fc.in_features, out_dim)
)


model.load_state_dict(model.state_dict())

model

Instalamos HiddenLayer para visualizar las gráficas de entrenamiento en tiempo real

In [ ]:
!pip install hiddenlayer

Y lo importamos

In [ ]:
import hiddenlayer as hl

hl.build_graph(model, torch.zeros([64,3,264,264]))

Finalmente, ponemos el modelo en modo de entrenamiento y entrenamos

In [ ]:
model = model.cuda()

model.train()

#--- Definimos nuestro criterio de evaluación y el optimizador 
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.1)
criterion = torch.nn.CrossEntropyLoss()


#--- Entrenamos el modelo usando únicamente 5 épocas
n_epochs = 5

history = hl.History()
canvas = hl.Canvas()

iter = 0

for epoch in range(n_epochs):
  for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_dl):
    # Pasamos os datos a 'cuda'
    
    X_train_batch = X_train_batch.cuda()
    Y_train_batch = Y_train_batch.cuda()

    # Realiza una predicción
    Y_pred = model(X_train_batch)

    # Calcula el loss
    loss = criterion(Y_pred, Y_train_batch)

    Y_pred = torch.argmax(Y_pred, 1)

    # Calcula el accuracy
    acc = sum(Y_train_batch == Y_pred)/len(Y_pred)
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if iter%10 == 0:
        #-- Visualizamos la evolución de los score loss y accuracy
        history.log((epoch+1, iter), loss=loss, accuracy=acc)
        with canvas:
          canvas.draw_plot(history["loss"])
          canvas.draw_plot(history["accuracy"])

    iter += 1
    del X_train_batch, Y_train_batch, Y_pred

Validamos finalmente el modelo con los datos de testeo

In [ ]:
from sklearn.metrics import f1_score


model.cpu()
model.eval()

Y_pred = model(X_test)
loss = criterion(Y_pred,Y_test)

Y_pred = torch.argmax(Y_pred, 1)
f1 = f1_score(Y_test, Y_pred, average='macro')

acc = sum(Y_test == Y_pred)/len(Y_pred)

print( 'Loss:{:.2f}, F1:{:.2f}, Acc:{:.2f}'.format(loss.item(), f1, acc ) )

Y lo guardamos para aprovecharlo eventualmente

In [ ]:
torch.save(model,open('./ResNet_MNIST.pt','wb'))